In [1]:
from requests import get
from requests.exceptions import RequestException
from contextlib import closing
import bs4
from bs4 import BeautifulSoup

import pandas as pd
import pickle

In [2]:
page = get('https://www.happyzebra.com/distance-calculator/farfrom-India.php')
soup = BeautifulSoup(page.content, 'html.parser')
root = soup.body
not_useful_ul = root.find_next('ul')
useful_ul = not_useful_ul.find_next('ul')
useful_list = list(useful_ul.children)

In [3]:
country_list = [str(i.find('a').contents[0])[9:].upper().split(" ") 
                for i in useful_ul 
                if(not type(i.find('a')) == int)]

In [4]:
for i in range(len(country_list)):
    if country_list[i][-1] == 'DISTANCE':
        country_list[i] = country_list[i][:-1]

In [5]:
country_list = [" ".join(i) for i in country_list]

In [6]:
distance_list = [str(i.contents[1]).split(" / ") for i in useful_ul
                 if (not type(i) == bs4.element.NavigableString)]

In [7]:
distance_miles = [float(distance_list[i][0].split(" ")[1]) for i in range(len(distance_list))]

In [8]:
distance_kms = [float(distance_list[i][1].split(" ")[0]) for i in range(len(distance_list))]

In [9]:
time_in_min = [int(distance_list[i][2].split(" ")[0]) * 60 + 
               int(distance_list[i][2].split(" ")[2]) 
               for i in range(len(distance_list))]

In [10]:
country_distance_data = pd.DataFrame([[country_list[i], distance_miles[i], 
                                       distance_kms[i], time_in_min[i]] 
                                      for i in range(len(country_list))], 
                                     columns = ['country_name', 'distance_miles', 
                                                'distance_kms', 'time_in_min'])

In [11]:
country_distance_data.to_pickle('./country_distance_data.pkl')
country_distance_data.to_csv('./country_distance_data.csv')